https://twitter.com/yujitach/status/1424030835771023363

In [1]:
using LinearAlgebra
using LoopVectorization
using LinearMaps
using ArnoldiMethod

function prepareDiag(L)
    diag = zeros(2^L)
    for state = 1:2^L
        for i = 1:L
            j = i==L ? 1 : i+1
            @inbounds diag[state] -= (((state >> (i-1))&1) == ((state >> (j-1))&1)) ? 1 : -1
        end
    end
    diag
end
    
function Hfunc!(C, B, diag, L)
    N = length(diag)
    @tturbo for state = 1:N
        C[state] = diag[state] * B[state]
    end
    for i = 1:L
        @tturbo for state = 1:N
            newstate = (state&(~(2^L))) ⊻ (1<<(i-1))
            c = newstate == 0
            newstate = !c*newstate + c*N # remove if statement
            C[newstate] -= B[state]
        end
    end
end
prepareHfunc!(L) = (C, B) -> Hfunc!(C, B, prepareDiag(L), L)

function f_LinearMap(L)
    H_LinearMap = LinearMap(prepareHfunc!(L), 2^L, ismutating=true, issymmetric=true, isposdef=false)
    d_LM, h_LM = partialschur(H_LinearMap, nev=1, which=SR())
    d_LM.eigenvalues[1]
end

f_exact(L) = -2sum(abs(sin((n-1/2) * pi/L)) for n in 1:L)

f_LinearMap(20), f_exact(20)

(-25.490989686364685 + 0.0im, -25.49098968636475)

In [2]:
@time f_LinearMap(20)
@time f_LinearMap(20)
@time f_LinearMap(20)

  7.019248 seconds (618 allocations: 968.031 MiB, 0.47% gc time)
  6.983104 seconds (615 allocations: 968.031 MiB, 1.10% gc time)
  7.001607 seconds (617 allocations: 968.031 MiB, 0.74% gc time)


-25.490989686364756 + 0.0im

https://juliaphysics.github.io/PhysicsTutorials.jl/tutorials/general/quantum_ising/quantum_ising.html

In [3]:
using LinearAlgebra
using SparseArrays
using ArnoldiMethod
⊗(x,y) = kron(x,y)

function TransverseFieldIsing_sparse(;N,h)
    id = [1 0; 0 1] |> sparse
    σˣ = [0 1; 1 0] |> sparse
    σᶻ = [1 0; 0 -1] |> sparse
    
    first_term_ops = fill(id, N)
    first_term_ops[1] = σᶻ
    first_term_ops[2] = σᶻ
    
    second_term_ops = fill(id, N)
    second_term_ops[1] = σˣ
    
    H = spzeros(Int, 2^N, 2^N)
    for i in 1:N
        H -= foldl(⊗, first_term_ops)
        first_term_ops = circshift(first_term_ops,1)
    end
    
    for i in 1:N
        H -= h*foldl(⊗, second_term_ops)
        second_term_ops = circshift(second_term_ops,1)
    end
    H
end

function f_sparse(L)
    H = TransverseFieldIsing_sparse(N=L, h=1)
    d, h = partialschur(H; nev=1, which=SR())
    d.eigenvalues[1]
end

f_exact(L) = -2sum(abs(sin((n-1/2) * pi/L)) for n in 1:L)

f_sparse(20), f_exact(20)

(-25.49098968636484 + 0.0im, -25.49098968636475)

In [4]:
L = 20
@time H = TransverseFieldIsing_sparse(N=L, h=1)
@time d, h = partialschur(H; nev=1, which=SR())

  3.051506 seconds (4.75 k allocations: 6.718 GiB, 14.31% gc time, 0.20% compilation time)
  6.839898 seconds (247 allocations: 328.026 MiB, 0.38% gc time, 0.22% compilation time)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([0.2360864632564727; 0.06481741680476345; … ; 0.06481741671756175; 0.23608646325357102;;], [-25.490989686364717;;], ComplexF64[-25.490989686364717 + 0.0im]), Converged: 1 of 1 eigenvalues in 80 matrix-vector products)

In [5]:
L = 20
@time H = TransverseFieldIsing_sparse(N=L, h=1)
@time d, h = partialschur(H; nev=1, which=SR())

  3.093728 seconds (4.66 k allocations: 6.718 GiB, 18.33% gc time)
  6.377327 seconds (68 allocations: 328.016 MiB)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([0.2360864633353528; 0.06481741687053828; … ; 0.06481741685256798; 0.2360864630598102;;], [-25.49098968636478;;], ComplexF64[-25.49098968636478 + 0.0im]), Converged: 1 of 1 eigenvalues in 80 matrix-vector products)

In [6]:
L = 20
@time H = TransverseFieldIsing_sparse(N=L, h=1)
@time d, h = partialschur(H; nev=1, which=SR())

  3.215592 seconds (4.66 k allocations: 6.718 GiB, 16.80% gc time)
  7.353999 seconds (64 allocations: 328.016 MiB)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([0.23608646317923557; 0.06481741678884152; … ; 0.06481741689138287; 0.2360864633477125;;], [-25.49098968636478;;], ComplexF64[-25.49098968636478 + 0.0im]), Converged: 1 of 1 eigenvalues in 80 matrix-vector products)

In [7]:
TransverseFieldIsing_sparse(N=20, h=1)

1048576×1048576 SparseMatrixCSC{Int64, Int64} with 21650584 stored entries:
⣿⣿⣾⢦⡀⠳⣄⠀⠀⠀⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠺⣟⢻⣶⣿⡂⠈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⢤⡈⠻⠻⠿⣧⣤⣠⡈⠳⠄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠙⢦⡀⠀⣻⣿⣿⣙⣦⡀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠙⢦⡈⠳⣼⣿⣿⡆⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠙⢦⡀⠀⠀⠁⠀⠈⠈⠉⣿⣿⣾⢦⡀⠳⣄⠀⠀⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠺⣟⢻⣶⣿⡂⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⢤⡈⠻⠻⠿⣧⣤⣠⡈⠳⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡀⠀⣻⣿⣿⣙⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠳⣼⣿⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄
⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⡟⢦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣍⣿⣿⣯⠀⠈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢦⡈⠋⠛⢻⣶⣦⣦⡈⠓⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠨⣿⠿⣧⣽⡦⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠀⠀⠙⢦⠈⠳⡿⣿⣿⣀⡀⡀⠀⢀⠀⠀⠈⠳⣄
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠸⣿⣿⡟⢦⡈⠳⣄⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠈⠻⣍⣿⣿⣯⠀⠈⠳⣄⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠐⢦⡈⠋⠛⢻⣶⣦⣦⡈⠓
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡀⠨⣿⠿⣧⣽⡦
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⠀⠀⠀⠙⢦⠈⠳⡿⣿⣿

In [8]:
function TransverseFieldIsing_sparse_revised(;N, h)
    id = [1 0; 0 1] |> sparse
    σˣ = [0 1; 1 0] |> sparse
    σᶻ = [1 0; 0 -1] |> sparse
    
    first_term_ops = fill(id, N)
    first_term_ops[1] = σᶻ
    first_term_ops[2] = σᶻ
    
    second_term_ops = fill(id, N)
    second_term_ops[1] = σˣ
    
    tmp = [spzeros(Int, 2^k, 2^k) for k in 1:N]
    H = spzeros(Int, 2^N, 2^N)
    
    for i in 1:N
        tmp[1] .= first_term_ops[mod1(i, N)]
        for k in 2:N
            kron!(tmp[k], tmp[k-1], first_term_ops[mod1(i+k-1, N)])
        end
        H .-= tmp[N]
    end
    
    for i in 1:N
        tmp[1] .= second_term_ops[mod1(i, N)]
        for k in 2:N
            kron!(tmp[k], tmp[k-1], second_term_ops[mod1(i+k-1, N)])
        end
        H .-= h .* tmp[N]
    end
    H
end

function f_sparse_revised(L)
    H = TransverseFieldIsing_sparse_revised(N=L, h=1)
    d, h = partialschur(H; nev=1, which=SR())
    d.eigenvalues[1]
end

TransverseFieldIsing_sparse_revised(N=10, h=1) == TransverseFieldIsing_sparse(N=10, h=1)

true

In [9]:
L = 20
@time H = TransverseFieldIsing_sparse_revised(N=L, h=1)
@time d, h = partialschur(H; nev=1, which=SR())

  2.718570 seconds (385 allocations: 4.078 GiB, 13.86% gc time)
  6.395660 seconds (68 allocations: 328.017 MiB)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([0.2360864631572488; 0.06481741670219336; … ; 0.06481741676573859; 0.2360864634248621;;], [-25.49098968636465;;], ComplexF64[-25.49098968636465 + 0.0im]), Converged: 1 of 1 eigenvalues in 80 matrix-vector products)

In [10]:
L = 20
@time H = TransverseFieldIsing_sparse_revised(N=L, h=1)
@time d, h = partialschur(H; nev=1, which=SR())

  2.812526 seconds (385 allocations: 4.078 GiB, 13.75% gc time)
  8.080196 seconds (65 allocations: 328.016 MiB, 2.54% gc time)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([0.23608646351658097; 0.0648174168264387; … ; 0.0648174167081197; 0.23608646301445615;;], [-25.490989686364827;;], ComplexF64[-25.490989686364827 + 0.0im]), Converged: 1 of 1 eigenvalues in 80 matrix-vector products)

In [11]:
L = 20
@time H = TransverseFieldIsing_sparse_revised(N=L, h=1)
@time d, h = partialschur(H; nev=1, which=SR())

  2.606386 seconds (385 allocations: 4.078 GiB, 5.99% gc time)
  6.393208 seconds (67 allocations: 328.016 MiB, 1.71% gc time)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([0.23608646323076649; 0.06481741678094972; … ; 0.06481741684356077; 0.2360864632854659;;], [-25.490989686364756;;], ComplexF64[-25.490989686364756 + 0.0im]), Converged: 1 of 1 eigenvalues in 80 matrix-vector products)

In [12]:
TransverseFieldIsing_sparse_revised(N=20, h=1)

1048576×1048576 SparseMatrixCSC{Int64, Int64} with 21650584 stored entries:
⣿⣿⣾⢦⡀⠳⣄⠀⠀⠀⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠺⣟⢻⣶⣿⡂⠈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⢤⡈⠻⠻⠿⣧⣤⣠⡈⠳⠄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠙⢦⡀⠀⣻⣿⣿⣙⣦⡀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠙⢦⡈⠳⣼⣿⣿⡆⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠙⢦⡀⠀⠀⠁⠀⠈⠈⠉⣿⣿⣾⢦⡀⠳⣄⠀⠀⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠺⣟⢻⣶⣿⡂⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⢤⡈⠻⠻⠿⣧⣤⣠⡈⠳⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡀⠀⣻⣿⣿⣙⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠳⣼⣿⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄
⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⡟⢦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣍⣿⣿⣯⠀⠈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢦⡈⠋⠛⢻⣶⣦⣦⡈⠓⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠨⣿⠿⣧⣽⡦⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠀⠀⠙⢦⠈⠳⡿⣿⣿⣀⡀⡀⠀⢀⠀⠀⠈⠳⣄
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠸⣿⣿⡟⢦⡈⠳⣄⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠈⠻⣍⣿⣿⣯⠀⠈⠳⣄⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠐⢦⡈⠋⠛⢻⣶⣦⣦⡈⠓
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡀⠨⣿⠿⣧⣽⡦
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⠀⠀⠀⠙⢦⠈⠳⡿⣿⣿

In [13]:
bit(a, k) = (a >> k) & 1

function sigmaz2(N)
    d = zeros(Int, 2^N)
    for k in 1:N, a in 0:2^N-1
        d[a+1] -= ifelse(bit(a, N-k) == bit(a, N - mod1(k+1, N)), 1, -1)
    end
    d
end

function sigmax(N, k)
    v = zeros(Int, 2^N - 2^(N-k))
    for i in 1:2:2^k-1
        v[(2^(N-k)*(i-1) + 1):2^(N-k)*i] .= -1
    end
    v
end

function TransverseFieldIsing_sparse_revised2(;N, h)
    d = sigmaz2(N)    
    v = sigmax.(N, 1:N)    
    H = spdiagm(
        ( 2^(N-k) => v[k] for k in 1:N)...,
        0 => d,
        (-2^(N-k) => v[k] for k in 1:N)...)
end

function f_sparse_revised2(L)
    H = TransverseFieldIsing_sparse_revised2(N=L, h=1)
    d, h = partialschur(H; nev=1, which=SR())
    d.eigenvalues[1]
end

TransverseFieldIsing_sparse_revised2(N=10, h=1) == TransverseFieldIsing_sparse(N=10, h=1)

true

In [14]:
L = 20
@time H = TransverseFieldIsing_sparse_revised2(N=L, h=1)
@time d, h = partialschur(H; nev=1, which=SR())

  2.278459 seconds (21.90 k allocations: 2.314 GiB, 0.70% gc time, 0.34% compilation time)
  9.104363 seconds (69 allocations: 328.017 MiB, 1.23% gc time)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([0.23608646328908184; 0.06481741673347335; … ; 0.06481741677941563; 0.2360864632506768;;], [-25.49098968636478;;], ComplexF64[-25.49098968636478 + 0.0im]), Converged: 1 of 1 eigenvalues in 80 matrix-vector products)

In [15]:
L = 20
@time H = TransverseFieldIsing_sparse_revised2(N=L, h=1)
@time d, h = partialschur(H; nev=1, which=SR())

  2.529292 seconds (485 allocations: 2.313 GiB, 13.89% gc time)
  9.387389 seconds (68 allocations: 328.016 MiB, 2.44% gc time)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([0.23608646298266253; 0.064817416713123; … ; 0.06481741687521565; 0.2360864635594257;;], [-25.49098968636476;;], ComplexF64[-25.49098968636476 + 0.0im]), Converged: 1 of 1 eigenvalues in 80 matrix-vector products)

In [16]:
L = 20
@time H = TransverseFieldIsing_sparse_revised2(N=L, h=1)
@time d, h = partialschur(H; nev=1, which=SR())

  2.528039 seconds (485 allocations: 2.313 GiB, 13.43% gc time)
  9.497895 seconds (68 allocations: 328.016 MiB, 2.19% gc time)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([0.23608646312626227; 0.06481741675487475; … ; 0.06481741671674497; 0.23608646334185027;;], [-25.4909896863647;;], ComplexF64[-25.4909896863647 + 0.0im]), Converged: 1 of 1 eigenvalues in 80 matrix-vector products)

In [17]:
function TransverseFieldIsing_sparse_revised2_time(;N, h)
    @time d = sigmaz2(N)    
    @time v = sigmax.(N, 1:N)    
    @time H = spdiagm(
        ( 2^(N-k) => v[k] for k in 1:N)...,
        0 => d,
        (-2^(N-k) => v[k] for k in 1:N)...)
end

TransverseFieldIsing_sparse_revised2_time(N=20, h=1); println()
TransverseFieldIsing_sparse_revised2_time(N=20, h=1); println()
TransverseFieldIsing_sparse_revised2_time(N=20, h=1)

  0.122171 seconds (2 allocations: 8.000 MiB)
  0.068290 seconds (41 allocations: 152.001 MiB, 42.27% gc time)
  2.354622 seconds (11.41 k allocations: 2.157 GiB, 13.18% gc time, 0.86% compilation time)

  0.119927 seconds (2 allocations: 8.000 MiB)
  0.043431 seconds (41 allocations: 152.001 MiB)
  2.534531 seconds (440 allocations: 2.156 GiB, 17.41% gc time)

  0.120879 seconds (2 allocations: 8.000 MiB)
  0.045864 seconds (41 allocations: 152.001 MiB)
  2.472143 seconds (440 allocations: 2.156 GiB, 17.97% gc time)


1048576×1048576 SparseMatrixCSC{Int64, Int64} with 40894466 stored entries:
⣿⣿⣾⣦⡀⠳⣄⠀⠀⠀⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠺⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⢤⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄
⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠓
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⡦
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⠀⠀⠀⠙⢦⠈⠻⡿⣿⣿

In [18]:
bit(a, k) = (a >> k) & 1

function sigmaz2_sparse(N)
    d = zeros(Int, 2^N)
    for k in 1:N, a in 0:2^N-1
        d[a+1] -= ifelse(bit(a, N-k) == bit(a, N - mod1(k+1, N)), 1, -1)
    end
    sparse(d) # should be sparse
end

function sigmax_sparse(N, k)
    v = zeros(Int, 2^N - 2^(N-k))
    for i in 1:2:2^k-1
        v[(2^(N-k)*(i-1) + 1):2^(N-k)*i] .= -1
    end
    sparse(v) # should be sparse
end

function TransverseFieldIsing_sparse_revised3(;N, h)
    d = sigmaz2_sparse(N)    
    v = sigmax_sparse.(N, 1:N)    
    H = spdiagm(
        ( 2^(N-k) => v[k] for k in 1:N)...,
        0 => d,
        (-2^(N-k) => v[k] for k in 1:N)...)
end

function f_sparse_revised3(L)
    H = TransverseFieldIsing_sparse_revised3(N=L, h=1)
    d, h = partialschur(H; nev=1, which=SR())
    d.eigenvalues[1]
end

TransverseFieldIsing_sparse_revised3(;N=10, h=1) == TransverseFieldIsing_sparse(;N=10, h=1)

true

In [19]:
L = 20
@time H = TransverseFieldIsing_sparse_revised3(N=L, h=1)
@time d, h = partialschur(H; nev=1, which=SR())

  2.053167 seconds (22.24 k allocations: 1.948 GiB, 24.06% gc time, 0.44% compilation time)
  6.359542 seconds (66 allocations: 328.016 MiB)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([0.23608646314766069; 0.06481741680829978; … ; 0.06481741684061726; 0.23608646333127445;;], [-25.490989686364745;;], ComplexF64[-25.490989686364745 + 0.0im]), Converged: 1 of 1 eigenvalues in 80 matrix-vector products)

In [20]:
L = 20
@time H = TransverseFieldIsing_sparse_revised3(N=L, h=1)
@time d, h = partialschur(H; nev=1, which=SR())

  1.963513 seconds (859 allocations: 1.947 GiB, 24.40% gc time)
  6.393544 seconds (66 allocations: 328.016 MiB, 2.26% gc time)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([-0.23608646317642293; -0.06481741663477773; … ; -0.0648174168382664; -0.23608646323938465;;], [-25.490989686364735;;], ComplexF64[-25.490989686364735 + 0.0im]), Converged: 1 of 1 eigenvalues in 80 matrix-vector products)

In [21]:
function TransverseFieldIsing_sparse_revised3_time(;N, h)
    @time d = sigmaz2_sparse(N)    
    @time v = sigmax_sparse.(N, 1:N)    
    @time H = spdiagm(
        ( 2^(N-k) => v[k] for k in 1:N)...,
        0 => d,
        (-2^(N-k) => v[k] for k in 1:N)...)
end

TransverseFieldIsing_sparse_revised3_time(N=20, h=1); println()
TransverseFieldIsing_sparse_revised3_time(N=20, h=1); println()
TransverseFieldIsing_sparse_revised3_time(N=20, h=1)

  0.125182 seconds (12 allocations: 25.879 MiB)
  0.315881 seconds (241 allocations: 456.943 MiB, 57.01% gc time)
  1.332958 seconds (11.56 k allocations: 1.476 GiB, 12.60% gc time, 1.79% compilation time)

  0.126099 seconds (12 allocations: 25.879 MiB)
  0.138541 seconds (241 allocations: 456.943 MiB)
  1.613559 seconds (604 allocations: 1.475 GiB, 21.87% gc time)

  0.124854 seconds (12 allocations: 25.879 MiB)
  0.377330 seconds (241 allocations: 456.943 MiB, 62.46% gc time)
  1.371604 seconds (604 allocations: 1.475 GiB, 13.39% gc time)


1048576×1048576 SparseMatrixCSC{Int64, Int64} with 21650584 stored entries:
⣿⣿⣾⢦⡀⠳⣄⠀⠀⠀⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠺⣟⢻⣶⣿⡂⠈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⢤⡈⠻⠻⠿⣧⣤⣠⡈⠳⠄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠙⢦⡀⠀⣻⣿⣿⣙⣦⡀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠙⢦⡈⠳⣼⣿⣿⡆⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠙⢦⡀⠀⠀⠁⠀⠈⠈⠉⣿⣿⣾⢦⡀⠳⣄⠀⠀⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠺⣟⢻⣶⣿⡂⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⢤⡈⠻⠻⠿⣧⣤⣠⡈⠳⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡀⠀⣻⣿⣿⣙⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠳⣼⣿⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄
⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⡟⢦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣍⣿⣿⣯⠀⠈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢦⡈⠋⠛⢻⣶⣦⣦⡈⠓⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠨⣿⠿⣧⣽⡦⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠀⠀⠙⢦⠈⠳⡿⣿⣿⣀⡀⡀⠀⢀⠀⠀⠈⠳⣄
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠸⣿⣿⡟⢦⡈⠳⣄⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠈⠻⣍⣿⣿⣯⠀⠈⠳⣄⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠐⢦⡈⠋⠛⢻⣶⣦⣦⡈⠓
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡀⠨⣿⠿⣧⣽⡦
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⠀⠀⠀⠙⢦⠈⠳⡿⣿⣿